# Customer Segmentation

### Import Packages

In [0]:
# !pip install xlrd
# import package
import pandas as pd
import datetime

# suppress error warnings
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

### Read Data

In [0]:
# load dataset
# data = pd.read_excel(Online Retail.xlsx')
data = pd.read_excel('Online Retail.xlsx')

# Define Sales Column
data['Sales'] = data['Quantity'] * data['UnitPrice']
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### Create RFM Segments

These are the the only 4 things we need to build our RFM segments:
1. *customers* : feature that specifies your users,
2. *dates* : dates of transactions
3. *transactions* : transaction number
4. *prices* : price of amount sold

In [0]:
data.shape

(541909, 9)

In [0]:
customers = 'CustomerID'
dates = 'InvoiceDate'
transactions = 'InvoiceNo'
prices = 'Sales'

In [0]:
#clean data

data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]

In [0]:
data.Country.drop_duplicates()

0               United Kingdom
26                      France
197                  Australia
385                Netherlands
1109                   Germany
1236                    Norway
1404                      EIRE
5320               Switzerland
6421                     Spain
6608                    Poland
7134                  Portugal
7214                     Italy
7279                   Belgium
7986                 Lithuania
9783                     Japan
14938                  Iceland
20000          Channel Islands
20017                  Denmark
29732                   Cyprus
30079                   Sweden
34083                  Finland
34293                  Austria
38313                  Bahrain
50791                   Israel
69007                   Greece
69623                Hong Kong
70758                Singapore
72985                  Lebanon
89570     United Arab Emirates
100810            Saudi Arabia
103598          Czech Republic
119191                  Canada
152712  

In [0]:
country = "USA"

# Filter for a single country
data_country = data[data.Country == country]

In [0]:
data = data_country

NOW = datetime.datetime.now()

# RFM Feature Engineering
rfmTable = data.groupby(customers).agg({dates: lambda x: (NOW - x.max()).days, transactions: lambda x: len(x), prices: lambda x: x.sum()})
rfmTable[dates] = rfmTable[dates].astype(int)
rfmTable.rename(columns={dates: 'recency', 
                         transactions: 'frequency', 
                         prices: 'monetary_value'}, inplace=True)

In [0]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12558.0,2933,11,269.96
12607.0,2986,101,1579.51
12646.0,2930,45,1346.97
12733.0,3160,22,383.95


In [0]:
rfmTable.shape

(4, 3)

### Segment Users Based on Quantiles 

The easiest way to split metrics into segments is by using quartiles.  

1. This gives us a starting point for the detailed analysis.
2. 4 segments are easy to understand and explain.


In [0]:
quantiles = rfmTable.describe()
quantiles

,recency,frequency,monetary_value
count,4.000000,4.000000,4.000000
mean,3002.250000,44.750000,895.097500
std,108.266261,40.086365,664.499282
min,2930.000000,11.000000,269.960000
25%,2932.250000,19.250000,355.452500
50%,2959.500000,33.500000,865.460000
75%,3029.500000,59.000000,1405.105000
max,3160.000000,101.000000,1579.510000


In [0]:
## RFM scorer
segmented_rfm = rfmTable.copy()

def RScore(x, p, d):
    if x <= d[p]['25%']:
        return 1
    elif x <= d[p]['50%']:
        return 2
    elif x <= d[p]['75%']: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p]['25%']:
        return 4
    elif x <= d[p]['50%']:
        return 3
    elif x <= d[p]['75%']: 
        return 2
    else:
        return 1

### Score Users

In [0]:
### Score each user
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency', quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

### Merge Scores
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm['RFMScore'] = segmented_rfm['RFMScore'].map(int)

segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12558.0,2933,11,269.96,2,4,4,244
12607.0,2986,101,1579.51,3,1,1,311
12646.0,2930,45,1346.97,1,2,2,122
12733.0,3160,22,383.95,4,3,3,433


In [0]:
data.to_csv('some_data.csv')

In [0]:

segmented_rfm.to_csv('some_file_name.csv')

files.download('some_file_name.csv')

## Identify Customer Segments

### Who are the top 10 of our best customers?  
RFM Score of *111*.

In [0]:
segmented_rfm['RFMScore'].sort_values().unique()

array([122, 244, 311, 433])

In [0]:
segmented_rfm[segmented_rfm['RFMScore']==111]

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,


### Who are our Loyal Customers?  
Frequency score of *1*.

In [0]:
segmented_rfm[segmented_rfm['f_quartile']==1].sort_values('frequency', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12607.0,2986,101,1579.51,3,1,1,311


### Who are our Big Spenders?  
Monetary value score of *1*.

In [0]:
segmented_rfm[segmented_rfm['m_quartile']==1].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12607.0,2986,101,1579.51,3,1,1,311


### Who are almost lost?  
Haven't purchased for some time but spent a lot and transacted a lot.  
RFM Score of *311*.

In [0]:
segmented_rfm[segmented_rfm['RFMScore']==311].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12607.0,2986,101,1579.51,3,1,1,311


### Who are lost?  
Haven't purchased in the longest time but spent a lot and transacted a lot.  
RFM score of *411*.

In [0]:
segmented_rfm[segmented_rfm['RFMScore']==411].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,


### Who are the bottom 10 worst customers?  
RFM score of *444*.

In [0]:
segmented_rfm[segmented_rfm['RFMScore']==444].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
